# Molecular classification of cancer

<b>Source of data:</b> 
Golub et al. (1999). <i>Molecular classification of cancer: class discovery and class prediction by gene expression monitoring</i>, Science, Vol. 286:531-537.

The data set golub consists of the expression levels of 3051 genes for 38 tumor mRNA samples. Each tumor mRNA sample comes from one patient (i.e. 38 patients total), and 27 of these tumor samples correspond to acute lymphoblastic leukemia (ALL) and the remaining 11 to acute myeloid leukemia (AML).

You will need to discover how many genes can be used to differentiate the tumor types (meaning that their expression level differs between the two tumor types) using

1. <b>The uncorrected p-values,

2. <b>The Holm-Bonferroni correction,
3. <b>The Benjamini-Hochberg correction</b>

In [28]:
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from scipy.stats import *
import pandas as pd

In [25]:
import numpy as np
import zipfile

with zipfile.ZipFile("statsreview_release.zip") as zip_file:
    golub_data, golub_classnames = ( np.genfromtxt(zip_file.open('data_and materials/golub_data/{}'.format(fname)), delimiter=',', names=True, converters={0: lambda s: int(s.strip(b'"'))}) for fname in ['golub.csv', 'golub_cl.csv'] )

In [30]:
data=pd.read_csv("data_and materials/golub_data/golub.csv", index_col=0).T
cl_names=pd.read_csv("data_and materials/golub_data/golub_cl.csv")
data.head()

,1,2,3,4,5,6,7,8,9,10,...,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051
V1,-1.45769,-0.75161,0.45695,3.13533,2.76569,2.64342,3.16885,2.88860,3.22372,3.22372,...,-0.31273,-0.69342,-0.30476,-0.21661,1.08935,0.04695,-0.20467,0.45231,-0.35920,-0.86079
V2,-1.39420,-1.26278,-0.09654,0.21415,-1.27045,1.01416,3.09954,2.95355,3.09954,3.09954,...,-0.45147,-0.80743,-0.72056,-0.65287,0.22701,0.48704,-0.07832,0.42686,-0.43633,-1.39420
V3,-1.42779,-0.09052,0.90325,2.08754,1.60433,1.70477,2.99977,2.99977,2.99977,2.99977,...,-0.76907,-0.51414,-0.11296,0.27332,0.31272,0.72170,-1.00615,0.67579,0.34031,-0.73766
V4,-1.40715,-0.99596,-0.07194,2.23467,1.53182,1.63845,3.28898,3.03972,3.34097,3.35455,...,-0.03863,-1.17554,-0.25346,-0.35475,0.47450,0.58403,-0.88748,0.31524,-0.90930,-1.19031
V5,-1.42668,-1.24245,0.03232,0.93811,1.63728,-0.36075,3.19368,3.21721,3.27515,3.27515,...,-0.99706,-1.42668,-0.99706,-0.89248,0.27257,0.30600,0.07175,-0.57779,-0.36663,-1.42668


In [31]:
data.shape

(38, 3051)

In [32]:
cl_names.head()

,Unnamed: 0,x
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [33]:
cl_names.shape

(38, 2)

In [35]:
cl_names.drop(columns='Unnamed: 0', inplace=True)

In [42]:
cl_names.iloc[26:]

,x
26,0
27,1
28,1
29,1
30,1
31,1
32,1
33,1
34,1
35,1


In [43]:
X_ALL= data.iloc[:27]
X_AML= data.iloc[27:]

In [44]:
X_AML

,1,2,3,4,5,6,7,8,9,10,...,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051
V28,-1.43337,-1.18065,0.26624,0.71990,0.02799,1.31110,3.18635,2.95821,3.16946,3.18635,...,-0.16468,-0.77111,-0.73237,-0.17376,-0.10777,0.40676,0.15568,0.65067,-0.49941,0.28711
V29,-1.08902,-1.08902,-0.43377,0.29598,-1.08902,-1.08902,3.56217,3.64076,3.04535,3.40936,...,-1.08902,-0.97512,-1.08902,-1.08902,-0.02267,0.39585,-0.13951,-0.13281,-1.08902,1.90481
V30,-1.29865,-1.05094,-0.10823,-1.29865,-1.29865,-1.29865,2.89050,2.79373,2.89050,2.89050,...,-0.18378,-0.37011,-1.02684,-0.76021,-0.69414,0.35006,-0.01776,0.35006,-0.62199,-0.21650
V31,-1.26183,-1.26183,-0.29385,2.76869,2.00518,1.73780,3.27934,3.25933,3.27934,3.27934,...,-0.61083,-1.01341,-0.22386,-0.26109,-0.05560,0.32939,-0.57690,0.32333,-0.11319,-1.00666
V32,-1.44434,-1.25918,0.05067,2.08960,1.17454,0.89347,3.06642,2.84412,3.09505,3.09505,...,-0.37646,-0.47674,-0.23164,-0.80229,0.28067,0.55791,-0.13350,0.75559,-0.51073,-0.46769
V33,1.10147,0.97813,1.69430,0.70003,-1.47218,-0.52883,2.88576,2.74728,2.88576,2.88576,...,-0.15436,-0.72762,-0.01620,-0.13063,-0.69228,0.11369,-0.33722,1.22578,-0.68539,0.30259
V34,-1.34158,-0.79357,-0.12472,0.13854,-1.34158,-1.22168,3.30283,3.17150,3.09109,3.27673,...,-0.38960,-0.11270,-0.22437,0.12390,0.04293,0.77377,-1.04023,0.59945,0.23005,-0.23523
V35,-1.22961,-1.22961,0.04609,1.75908,1.55086,0.90832,3.30751,2.91010,3.20545,3.49711,...,0.04086,-0.42929,-0.63223,0.15231,0.09446,-0.05275,-0.71434,0.72956,-0.17911,1.43690
V36,-0.75919,-0.71792,0.24347,0.06151,-1.18107,-1.39906,2.81781,2.81781,2.81781,2.81781,...,-0.27681,-0.34404,-0.33015,-0.18327,-0.22265,0.93171,-0.08540,0.72762,-0.37737,-0.65113
V37,0.84905,0.45127,0.90774,1.30297,1.01596,0.51266,3.01414,2.96104,3.01414,3.01414,...,-0.63777,-0.76331,-0.70433,-0.56208,-0.17003,0.01243,-0.30978,1.10466,-0.39063,1.62964


In [46]:
X_bar_ALL = X_ALL.mean(axis=0)
X_bar_AML = X_AML.mean(axis=0)
X_bar_AML

1      -0.779247
2      -0.691942
3       0.246639
4       0.829046
5      -0.040917
          ...   
3047    0.425924
3048   -0.312812
3049    0.682165
3050   -0.354276
3051    0.416756
Length: 3051, dtype: float64

In [47]:
S_ALL = X_ALL.var(ddof=1)
S_AML = X_AML.var(ddof=1)

In [49]:
N_ALL = 27
N_AML = 11
S_Xbar_ALL = S_ALL/N_ALL
S_Xbar_AML = S_AML/N_AML

In [50]:
t_Welch = (X_bar_ALL - X_bar_AML)/np.sqrt(S_Xbar_ALL + S_Xbar_AML)
t_Welch

1      -1.759195
2      -0.909858
3       0.098026
4       0.338963
5       1.370165
          ...   
3047   -0.041362
3048   -0.316234
3049   -1.842529
3050   -0.104688
3051   -3.292583
Length: 3051, dtype: float64

In [95]:
nu = np.square(S_Xbar_ALL + S_Xbar_AML)/( np.square(S_Xbar_ALL)/(N_ALL-1) + np.square(S_Xbar_AML)/(N_AML-1))
nu

1       11.048864
2       12.511904
3       14.982441
4       31.303943
5       26.178243
          ...    
3047    18.014322
3048    35.410665
3049    21.283981
3050    23.913221
3051    12.258054
Length: 3051, dtype: float64

In [98]:
stat, p_val = ttest_ind(X_ALL, X_AML)

In [99]:
np.sum(p_val<=0.05)

1045

In [115]:
np.sum((np.flip(t_Welch.index)*p_val)<=0.05)

98

In [109]:
p_val.sort()

In [116]:
a = np.array([1,2,3])

In [117]:
3/a

array([3. , 1.5, 1. ])

In [114]:
np.sum( (3051*p_val/t_Welch.index) <=0.05)

681